In [17]:
import pandas as pd

# Original dataframe
df = pd.read_excel("V:\\Retail\\RETAIL PERFORMANCE_EXCELLENCE\\18. Projects\\2023\\06.05 ROckstud Incetive\\db\\Rockstud_incentive_db.xlsx")
df = df.rename(columns={"1": "Continuative SKU"})


def re_code(id):
    if id == 'MEXICO CITY POLANCO SHOES':
        return 'MEXICO CITY POLANCO'
    else:
        return id
    
df["Store Descr."] = df['Store Descr.'].apply(re_code)
df.head

<bound method NDFrame.head of            Date              Store Descr.  Key_Transactions Line Reclass  \
0    2023-06-09            COSTA MESA SCP           6655760    VGD SHOES   
1    2023-06-09            COSTA MESA SCP           6657256    VGD SHOES   
2    2023-06-09            COSTA MESA SCP           6657257    VGD SHOES   
3    2023-06-09            COSTA MESA SCP           6658453    VGD SHOES   
4    2023-06-09            COSTA MESA SCP           6661737     VGD BAGS   
...         ...                       ...               ...          ...   
3501 2023-06-05                     ASPEN           6679003     VGD BAGS   
3502 2023-06-05                     ASPEN           6688048     VGD BAGS   
3503 2023-06-05        NEW YORK SAKS BAGS           6682033     VGD BAGS   
3504 2023-06-05  DALLAS NM NORTHPARK BAGS           6688011     VGD BAGS   
3505 2023-06-05      ATLANTA PHIPPS PLAZA           6680724    VGD SHOES   

     Key Associate Continuative SKU  TY MTD SALES QTY  
0

In [18]:
from collections import defaultdict 

# Filter rows where 'SKU' column contains 'Ballerinas'
Ballerinas = set(['WS0HQ2EQL0NO', 'WS0HQ2SWHGF9', 'WS0HB6ETNKDZ', 'WS0HQ5VNWGF9', 'WS0HB6EQL0NO', 'WS0HB6SWHGF9', 'WS0HB6SWH6E0', 'WS0HB6MJUUWT'])
Rockstud_Handbags = set(['WB0M41QYV0NO', 'WB0M41AZSUWT', 'WB0M41AZS098', 'WB0M42AZSUWT', 'WB0M42AZS098', 'WB0M42QTES13'])

#df["Key Associate"] = df[df["Key Associate"] == 'BERND2']

# Create a transactiosn dataframe and compute the points
transactions = df.groupby(["Key_Transactions", "Store Descr.", "Date"]).agg(
    {"Line Reclass": list, "Continuative SKU": list, "TY MTD SALES QTY": list}
)

def compute_points(line_reclass_list, sku_list, qty_list):
    """Compute points for a transaction."""
    points = 0
    has_shoes = False
    has_bags = False
    for item, sku, quantity in zip(line_reclass_list, sku_list, qty_list):
        if 'VGD SHOES' in item:  # Check for 'VGD SHOES' in item
            points += quantity * 2
            has_shoes = True
        elif 'VGD BAGS' in item:  # Check for 'VGD BAGS' in item
            points += quantity
            has_bags = True
        # Check if sku is in either set and add points
        if sku in Ballerinas or sku in Rockstud_Handbags:
            points += 1
    # Check if both shoes and bags are present in transaction and add an extra point
    if has_shoes and has_bags:
        points += 1
    # Make sure points are not negative
    return max(0, points)

def cs(line_reclass_list, date):
    """Check if a transaction contains both VGD SHOES and VGD BAGS within a date range."""
    start_date = pd.Timestamp('2023-06-01')
    end_date = pd.Timestamp('2023-06-30')
    transaction_date = pd.Timestamp(date)
    if start_date <= transaction_date <= end_date and 'VGD SHOES' in line_reclass_list and 'VGD BAGS' in line_reclass_list:
        return 1
    return 0

transactions = df.groupby(["Key_Transactions", "Store Descr.", "Date"]).agg(
    {"Line Reclass": list, "Continuative SKU": list, "TY MTD SALES QTY": list}
)

#transactions["cs"] = transactions.apply(lambda row: cs(row["Line Reclass"], row.name[2]), axis=1)
transactions["points"] = transactions.apply(
    lambda row: compute_points(row["Line Reclass"], row["Continuative SKU"], row["TY MTD SALES QTY"]), axis=1
)

transactions

#transactions.xs(('TORONTO YORKDALE', '2023-06-17'), level=('Store Descr.', 'Date'))

,,,Line Reclass,Continuative SKU,TY MTD SALES QTY,points
Key_Transactions,Store Descr.,Date,,,,
6649687,BAL HARBOUR,2023-06-16,[VGD SHOES],[WS0393KFDMNG],[1],2
6649688,BAL HARBOUR,2023-06-20,[VGD SHOES],[WS0393KFDMNG],[-1],0
6649689,BAL HARBOUR,2023-06-03,[VGD SHOES],[WS0393VNWP45],[1],2
6649690,BAL HARBOUR,2023-06-16,[VGD SHOES],[WS0393VNWP45],[-1],0
6649701,NEW YORK MADISON AV.,2023-06-13,[VGD SHOES],[WS0F95BAE0NO],[-1],0
...,...,...,...,...,...,...
11606867,NEW YORK SAKS BAGS,2023-06-16,[VGD BAGS],[WB0M37TAG0NO],[1],1
11610542,SEATTLE NORDSTROM BAGS,2023-06-10,[VGD BAGS],[WB0M37TAG0NO],[1],1
11610772,SEATTLE NORDSTROM BAGS,2023-06-10,[VGD BAGS],[WB0M37TAG0NO],[-1],0


In [19]:
dashboard = transactions.groupby(["Store Descr.", "Date"]).sum()
dashboard

points
Store Descr.     Date              
ASPEN            2023-06-02       2
                 2023-06-05       2
                 2023-06-08       2
                 2023-06-09       2
                 2023-06-10       3
...                             ...
TORONTO YORKDALE 2023-06-26       8
                 2023-06-27      14
                 2023-06-28      15
                 2023-06-29      13
                 2023-06-30      14

[714 rows x 1 columns]

In [20]:
dashboard.xs("TORONTO YORKDALE",level="Store Descr.")

,points
Date,
2023-06-01,16
2023-06-02,15
2023-06-03,14
2023-06-04,24
2023-06-05,12
2023-06-06,27
2023-06-07,7
2023-06-08,13
2023-06-09,23


In [25]:
#average_sales_qty = pd.read_excel("V:\\Retail\\RETAIL PERFORMANCE_EXCELLENCE\\18. Projects\\2023\\06.05 ROckstud Incetive\\db\\AVG_WEEKDAY.xlsx")

def assign_points(store):
    store = store.upper()
    if store == 'COSTA MESA SCP':
        return 40
    elif store == 'LAS VEGAS FORUM':
        return 30
    elif store == 'PALM BEACH WORTH AV':
        return 20
    elif store == 'BAL HARBOUR':
        return 60
    elif store == 'LAS VEGAS BELLAGIO':
        return 50
    elif store == 'NEW YORK SAKS BAGS':
        return 40
    else:
        return 0

# Create a dictionary that maps stores to groups
store_groups = {'ASPEN': 'C', 'ATLANTA PHIPPS PLAZA': 'C', 'AVENTURA MALL TEMP': 'B', 'BAL HARBOUR': 'A', 'COSTA MESA SCP': 'A', 'MIAMI DESIGN DISTRICT': 'A', 'DALLAS HIGHLAND PARK VILLAGE': 'B', 'TORONTO YORKDALE': 'A', 'LAS VEGAS BELLAGIO': 'B', 'NEW YORK SOHO': 'A', 'BEVERLY HILLS': 'A', 'BOSTON NEWBURY ST.': 'B', 'LAS VEGAS THE SHOPS CRYSTALS': 'B', 'HOUSTON THE GALLERIA': 'B', 'LAS VEGAS FORUM': 'B', 'SAN FRANCISCO GRANT AV.': 'C', 'HONOLULU ALA MOANA': 'A', 'HONOLULU R. HAWAIIAN CENTER':'A', 'PALM BEACH WORTH AV.': 'C', 'NEW YORK MADISON AV.': 'C', 'NEW YORK SAKS BAGS': 'C', 'CANCUN': 'B', 'ATLANTA PHIPPS PLAZA': 'C', 'MANHASSET HIRSHLEIFERS': 'C', 'ASPEN': 'C',
'MEXICO CITY PERISUR': 'B', 'DALLAS NM NORTHPARK BAGS': 'C', 'BEVERLY HILLS NM BAGS': 'C', 'MEXICO CITY POLANCO':'A', 'SEATTLE NORDSTROM BAGS': 'C'
}

def highlight_qualifier(row):
    styles = []
    for column, value in row.iteritems():
        if column == "cs" and value > 0:
            styles.append("background-color: lightgreen")
        else:
            styles.append("")  # No background
    return styles

total = dashboard.groupby("Store Descr.").sum()

# Add a column to the DataFrame that maps stores to groups
total["group"] = [store_groups.get(g) for g in total.index]

# Add the assigned points to the "points" of each store
total["points"] += [assign_points(g) for g in total.index]

# Sort and filter the DataFrame first
total = total.sort_values(by=["group", "points"], ascending=[True, False])[["points", "group"]]

# Then apply the style and display the result
styled_total = total.style.apply(highlight_qualifier, axis=1)
#display(styled_total)

# Add the average sales quantities to the total DataFrame
#total_with_sales_qty = pd.merge(total, average_sales_qty, on='Store Descr.', how='left')
#total_with_sales_qty = total.join(average_sales_qty, how='left', rsuffix='_avg_sales')

# Rename the column for clarity
#total_with_sales_qty = total_with_sales_qty.rename(columns={'TY MTD SALES QTY': 'Week Day QTY avg'})
#total_with_sales_qty['Average'] = total_with_sales_qty['Average'].astype(int)

# Apply the style and display the result
#styled_total_with_sales_qty = total_with_sales_qty.style.apply(highlight_qualifier, axis=1)
display(styled_total)


,points,group
Store Descr.,,
COSTA MESA SCP,478,A
TORONTO YORKDALE,425,A
MEXICO CITY POLANCO,412,A
HONOLULU ALA MOANA,329,A
HONOLULU R. HAWAIIAN CENTER,296,A
BAL HARBOUR,278,A
NEW YORK SOHO,278,A
BEVERLY HILLS,275,A
MIAMI DESIGN DISTRICT,258,A


In [26]:
hr_df = pd.read_excel("C:\\Users\\cadae\\OneDrive - Valentino S.p.A\\HR Retail\\Retail Headcount Active & Inactive.xlsx")

# Group by 'Key Associate' and 'Line Reclass', and aggregate the sum of 'TY MTD SALES QTY'
products_df = df.groupby(['Key Associate', 'Line Reclass'])['TY MTD SALES QTY'].sum().reset_index()

# Pivot the DataFrame to get 'VGD BAGS' and 'VGD SHOES' as separate columns
products_df = products_df.pivot(index='Key Associate', columns='Line Reclass', values='TY MTD SALES QTY').reset_index()

# Fill NaN values with 0 if any
products_df = products_df.fillna(0)

# Group by 'Key Associate' and 'Store Descr.', calculate the sum of 'TY MTD SALES QTY' per store per associate
grouped_store_df = df.groupby(["Key Associate", "Store Descr."])["TY MTD SALES QTY"].sum().reset_index()

# Order by 'TY MTD SALES QTY', descending, so the first entry for each "Key Associate" will be their highest scoring store
grouped_store_df = grouped_store_df.sort_values(["Key Associate", "TY MTD SALES QTY"], ascending=[True, False])

# Drop duplicates of 'Key Associate', keeping the first entry (which is the highest scoring due to our sorting)
grouped_store_df = grouped_store_df.drop_duplicates(subset='Key Associate')

# Group by 'Key Associate' to get total sales per associate
grouped_df = df.groupby(["Key Associate"])["TY MTD SALES QTY"].sum().reset_index()

# Merge the dataframes on 'Key Associate' to get the store each Key Associate belongs to (the store where they had most transactions)
grouped_df = pd.merge(grouped_df, grouped_store_df[['Key Associate', 'Store Descr.']], on='Key Associate', how='left')

# Group by 'Key Associate' and 'Store Descr.', calculate the sum of 'TY MTD SALES QTY'
#grouped_df = df.groupby(["Key Associate", "Store Descr."])["TY MTD SALES QTY"].sum().reset_index()

# Order by 'TY MTD SALES QTY', descending, so the first entry for each "Key Associate" will be their highest scoring store
#grouped_df = grouped_df.sort_values(["Key Associate", "TY MTD SALES QTY"], ascending=[True, False])

# Drop duplicates of 'Key Associate', keeping the first entry (which is the highest scoring due to our sorting)
#grouped_df = grouped_df.drop_duplicates(subset='Key Associate')

# If you want to sort the dataframe back by 'Date'
#grouped_df = grouped_df.sort_values('Date').reset_index(drop=True)


def assign_value(group):
    if group == 'A':
        return 15
    elif group == 'B':
        return 12
    elif group == 'C':
        return 5
    else:
        return 0  # or any default value you want to assign

def re_coding(id):
    if id == 'ZHANZ':
        return 'ZOEY'
    elif id == 'NALAB':
        return 'BAYINN'
    elif id == 'MANUE':
        return 'EMANUEL'
    elif id == 'HOTTM':
        return 'MIYAKOH'
    elif id == 'baiis':
        return 'BAIIS'
    elif id == 'DAVIK':
        return 'KENT'
    elif id == 'benea1':
        return 'BENEA1'
    elif id == 'youkn':
        return 'YOUKN'
    elif id == 'escub':
        return 'ESCUB'
    elif id == 'GORLOV':
        return 'GABRIELA'
    elif id == 'HARBV':
        return 'VERONICA'
    elif id == 'felij':
        return 'FELIJ'
    elif id == 'moctl':
        return 'MOCTL'
    elif id == 'e_Roaom':
        return 'E_ROAOM'
    elif id == 'garcj1':
        return 'GARCJ1'
    elif id == 'marqr':
        return 'MARQR'
    elif id == 'BARTG2':
        return 'GIANNA'
    elif id == 'herrh':
        return 'HERRH'
    elif id == 'NEVORA':
        return 'NEUSA'
    elif id == 'PRODN':
        return 'NINA'
    elif id == 'LEGWE':
        return 'GWEN'
    else:
        return id

def highlight_qualifier(row):
    styles = []
    for column, value in row.iteritems():
        if column == "Qualified" and value == "YES":
            styles.append("background-color: lightgreen")
        else:
            styles.append("")  # No background
    return styles

#recording
hr_df["Username"] = hr_df['Username'].apply(re_coding)

# Create a 'gift' column
grouped_df["Gift card value"] = grouped_df["TY MTD SALES QTY"] * 25

# Add a 'group' column based on the 'Store Descr.' column
grouped_df["Group"] = [store_groups.get(g) for g in grouped_df["Store Descr."]]

# Add a new column based on the 'group' column
grouped_df['Qualifier'] = grouped_df['Group'].apply(assign_value)

# Add a column 'Qualified' based on whether 'TY MTD SALES QTY' is greater than 'qualifier'
grouped_df['Qualified'] = grouped_df.apply(lambda row: 'YES' if row['TY MTD SALES QTY'] >= row['Qualifier'] else 'NO', axis=1)

# Sort the DataFrame by 'group' (ascending) and 'gift' (descending)
grouped_df = grouped_df.sort_values(by=["Group", "Gift card value"], ascending=[True, False])

# Merge on 'Key Associate' (assuming different names in grouped_df and hr_df)
merged_df = pd.merge(grouped_df, hr_df[['Username', 'Last Name', 'First Name', 'Title']], left_on='Key Associate', right_on='Username', how='left')

# Merge on 'Key Associate' with the products_df
merged_df = pd.merge(merged_df, products_df, on='Key Associate', how='left')

# Remove rows where Title is 'Store Manager' or 'Team Manager'
merged_df = merged_df[~merged_df['Title'].isin(['Store Manager', 'Team Manager'])]

# Keep the columns you need
merged_df = merged_df[["Key Associate", "Store Descr.", "Last Name", "First Name", "TY MTD SALES QTY", "Qualifier", "Qualified", "Gift card value", "Group", "VGD BAGS", "VGD SHOES"]]

# Convert to integer
merged_df['VGD BAGS'] = merged_df['VGD BAGS'].astype(int)
merged_df['VGD SHOES'] = merged_df['VGD SHOES'].astype(int)

merged_df = merged_df.loc[merged_df["TY MTD SALES QTY"] > 0]

# Sort DataFrame by 'Gift card value' in descending order
merged_df = merged_df.sort_values(by="Gift card value", ascending=False)

merged_df = merged_df.style.apply(highlight_qualifier, axis=1)

# Print DataFrame
merged_df


,Key Associate,Store Descr.,Last Name,First Name,TY MTD SALES QTY,Qualifier,Qualified,Gift card value,Group,VGD BAGS,VGD SHOES
77,BERND2,LAS VEGAS FORUM,Bernal,Denys,64,12,YES,1600,B,5,59
78,ZOEY,LAS VEGAS FORUM,Zhang,Jing Jing,64,12,YES,1600,B,9,55
0,ARVIV,MEXICO CITY POLANCO,ARVIZU RUBIO,VICTOR HUGO,57,15,YES,1425,A,2,55
79,PEREJ1,LAS VEGAS THE SHOPS CRYSTALS,Perez,Jiayi,52,12,YES,1300,B,9,43
80,ZAGOB,LAS VEGAS FORUM,Zagorska,Bilyana,50,12,YES,1250,B,5,45
1,HIGAL,HONOLULU ALA MOANA,Higaki,Lisa,48,15,YES,1200,A,7,41
81,ALVAM1,LAS VEGAS BELLAGIO,Alva,Matthew,45,12,YES,1125,B,8,37
2,HERNM,COSTA MESA SCP,Hernandez,Bruce,42,15,YES,1050,A,7,35
3,ZEGAM,MIAMI DESIGN DISTRICT,Zegara,Michelle,41,15,YES,1025,A,4,37
82,NGUYV,BOSTON NEWBURY ST.,Nguyen,Vu,40,12,YES,1000,B,7,33


In [24]:
import os


def save_xlsx(df, folder, name):
    filename = f"{folder}/{name}.xlsx"
    # os.makedirs(os.path.dirname(filename), exist_ok=True)
    with pd.ExcelWriter(filename) as writer:
        df.to_excel(writer, index=False)

save_xlsx(merged_df, "V:\\Retail\\RETAIL PERFORMANCE_EXCELLENCE\\18. Projects\\2023\\06.05 ROckstud Incetive\\db", "ca_example")